 **To**: Megaline Management

 **From**: Junior Data Scientist

 **Date**: August 8, 2025
 
 **Subject**: Preliminary Analysis of Surf and Ultimate Prepaid Plans

# Introduction
This report presents a preliminary analysis of Megaline's 'Surf' and 'Ultimate' prepaid plans. The primary goal is to determine which of these two plans generates more revenue. By analyzing the behavior of a sample of 500 customers from 2018, we can gain insights that will help the commercial department make informed decisions about the allocation of the advertising budget.

# 1. Data Loading and Initial Exploration
First, let's load all the necessary libraries and the datasets to get a first look at the data we're working with.

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats as st
import os

try:
    calls = pd.read_csv('data/megaline_calls.csv')
    internet = pd.read_csv('data/megaline_internet.csv')
    messages = pd.read_csv('data/megaline_messages.csv')
    plans = pd.read_csv('data/megaline_plans.csv')
    users = pd.read_csv('data/megaline_users.csv')
except FileNotFoundError as e:
    print(f"Error: {e}. Make sure all CSV files are in the same directory.")

print("Initial Data Info:")
users.info()
calls.info()
messages.info()
internet.info()
plans.info()

Initial Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     500 non-null    int64 
 1   first_name  500 non-null    object
 2   last_name   500 non-null    object
 3   age         500 non-null    int64 
 4   city        500 non-null    object
 5   reg_date    500 non-null    object
 6   plan        500 non-null    object
 7   churn_date  34 non-null     object
dtypes: int64(2), object(6)
memory usage: 31.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137735 entries, 0 to 137734
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   id         137735 non-null  object 
 1   user_id    137735 non-null  int64  
 2   call_date  137735 non-null  object 
 3   duration   137735 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 4.2+ MB
<class '

### **Initial Observations**
- Data Types: Several date columns (reg_date, churn_date, call_date, message_date, session_date) are currently stored as object data types. These will need to be converted to a proper datetime format for time-based analysis.

- Missing Values: The churn_date column in the users table has a significant number of missing values. The project description states that if the value is missing, the plan was still in use at the time of data extraction. This is expected and doesn't represent an error.

- Call Duration: The duration in the calls table is a float. The plan details specify that call durations are rounded up to the nearest minute for billing. I also note the presence of zero-duration calls, which could represent missed or dropped calls. These still consume resources to connect, so they should be investigated.

- Data Volume: Internet usage (mb_used) is given in megabytes. For billing, the total monthly data usage is rounded up to the next gigabyte. This will need to be calculated.

# 2. Data Preparation
In this step, I will clean and transform the data to make it suitable for analysis. This includes converting data types, creating a month column for aggregation, and rounding values according to the plan rules.

I will then aggregate the data to calculate the monthly usage for each customer and merge all the information into a single, comprehensive DataFrame. Finally, I will calculate the monthly revenue generated by each user.

In [7]:
# Convert date columns to datetime objects
users['reg_date'] = pd.to_datetime(users['reg_date'], format='%Y-%m-%d')
users['churn_date'] = pd.to_datetime(users['churn_date'], format='%Y-%m-%d')
calls['call_date'] = pd.to_datetime(calls['call_date'], format='%Y-%m-%d')
messages['message_date'] = pd.to_datetime(messages['message_date'], format='%Y-%m-%d')
internet['session_date'] = pd.to_datetime(internet['session_date'], format='%Y-%m-%d')

# Round up call durations and convert to integer
calls['duration'] = np.ceil(calls['duration']).astype(int)

# Add a 'month' column
calls['month'] = calls['call_date'].dt.month
messages['month'] = messages['message_date'].dt.month
internet['month'] = internet['session_date'].dt.month

# Aggregate data per user per month
calls_agg = calls.groupby(['user_id', 'month']).agg(total_calls=('id', 'count'), total_minutes=('duration', 'sum')).reset_index()
messages_agg = messages.groupby(['user_id', 'month']).agg(total_messages=('id', 'count')).reset_index()
internet_agg = internet.groupby(['user_id', 'month']).agg(total_mb_used=('mb_used', 'sum')).reset_index()

# Merge aggregated data
merged_data = pd.merge(calls_agg, messages_agg, on=['user_id', 'month'], how='outer')
merged_data = pd.merge(merged_data, internet_agg, on=['user_id', 'month'], how='outer')

# Merge with user and plan information
merged_data = pd.merge(merged_data, users, on='user_id', how='left')
plans.rename(columns={'plan_name': 'plan'}, inplace=True)
merged_data = pd.merge(merged_data, plans, on='plan', how='left')

# Fill NaN values with 0 for usage columns
usage_cols = ['total_calls', 'total_minutes', 'total_messages', 'total_mb_used']
for col in usage_cols:
    merged_data[col] = merged_data[col].fillna(0)

# Convert mb to gb for plan inclusion and usage
merged_data['gb_per_month_included'] = merged_data['mb_per_month_included'] / 1024
merged_data['total_gb_used'] = np.ceil(merged_data['total_mb_used'] / 1024)

# Function to calculate monthly revenue
def calculate_revenue(row):
    plan_monthly_fee = row['usd_monthly_pay']
    minutes_over = max(0, row['total_minutes'] - row['minutes_included'])
    minutes_overage_cost = minutes_over * row['usd_per_minute']
    messages_over = max(0, row['total_messages'] - row['messages_included'])
    messages_overage_cost = messages_over * row['usd_per_message']
    gb_over = max(0, row['total_gb_used'] - row['gb_per_month_included'])
    gb_overage_cost = gb_over * row['usd_per_gb']
    return plan_monthly_fee + minutes_overage_cost + messages_overage_cost + gb_overage_cost

merged_data['revenue'] = merged_data.apply(calculate_revenue, axis=1)
print("Data preparation complete.")

Data preparation complete.


The data is now clean, merged, and ready for analysis. The merged_data DataFrame contains all the necessary information, including user details, plan specifics, monthly usage, and the calculated monthly revenue.

# 3. Data Analysis
Now, I will analyze the prepared data to understand the behavior of customers on the 'Surf' and 'Ultimate' plans. I will calculate the mean, variance, and standard deviation of their monthly usage (minutes, messages, and data) and then create histograms to visualize and compare the distributions.

In [ ]:
plan_analysis = merged_data.groupby('plan').agg(
    mean_revenue=('revenue', 'mean'),
    median_revenue=('revenue', 'median'),
    mean_minutes=('total_minutes', 'mean'),
    std_dev_minutes=('total_minutes', 'std'),
    var_minutes=('total_minutes', 'var'),
    mean_messages=('total_messages', 'mean'),
    std_dev_messages=('total_messages', 'std'),
    var_messages=('total_messages', 'var'),
    mean_gb=('total_gb_used', 'mean'),
    std_dev_gb=('total_gb_used', 'std'),
    var_gb=('total_gb_used', 'var')
)
print("--- Plan Usage Analysis ---")
print(plan_analysis)

# Create a directory for plots if it doesn't exist
if not os.path.exists('plots'):
    os.makedirs('plots')

# Plot histograms
for metric in ['total_minutes', 'total_messages', 'total_gb_used', 'revenue']:
    plt.figure(figsize=(10, 6))
    for plan in merged_data['plan'].unique():
        subset = merged_data[merged_data['plan'] == plan]
        plt.hist(subset[metric], bins=30, alpha=0.7, label=plan)
    title_metric = metric.replace('_', ' ').title()
    plt.title(f'Distribution of Monthly {title_metric} per Plan')
    plt.xlabel(title_metric)
    plt.ylabel('Frequency')
    plt.legend()
    plt.grid(True)
    plt.savefig(f'plots/histogram_{metric}.png')
    plt.close() # Close the plot to avoid displaying it in the console
print("Histograms saved in 'plots' directory.")

IndentationError: unexpected indent (1978462413.py, line 14)